In [1]:
import numpy as np
from utils_clean import generate_sample, DEOptim, RandomOptim, PSOOptim, visualize_meshgrid, get_likelihood_grids, visualize_pred_meshgrid
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
import plotly.express as px
import pandas as pd

In [2]:
target_func = lambda X: (20 + X**2 - 10 * np.cos(2*np.pi*X)).sum(axis=0)

n_dims = 2
lower_bound = -2.5, -1.5
upper_bound = +1.5, +2.5

In [3]:
de = DEOptim(visualize=False, maxiter=100, popsize=10)

In [4]:
rs = RandomOptim(maxiter=1000, visualize=False)

In [5]:
seed = np.random.RandomState(42)
part = 10
dim = 2
pso = PSOOptim(n_iters=100, n_particles=part, c1=0.5, c2=0.3, w=0.9, init_pos=(seed.rand(part, dim)*22)-11, visualize=False)

In [6]:
results_dict = {
    "optim": [],
    "n": [],
    "log_likelihood": [],
    # "MSE": []
}

In [7]:
from tqdm import tqdm

In [8]:

for optim in tqdm([de, rs, pso]):
    for n in [10, 70, 500, 1000, 5000]:
        for _ in range(20):
            gpr = GaussianProcessRegressor(optimizer=optim.optimize, kernel=RBF(length_scale=(1, 1)))
            X, y = generate_sample(n, n_dims, lower_bound, upper_bound, target_func)
            gpr.fit(X.T, y[...,None])
            results_dict["optim"].append(optim.__class__.__name__)
            results_dict["n"].append(n)
            results_dict["log_likelihood"].append(gpr.log_marginal_likelihood_value_)
            # results_dict["MSE"].append(mean_squared_error(y, gpr.predict(X.T)))

  0%|          | 0/3 [22:52<?, ?it/s]


In [ ]:
df = pd.DataFrame(results_dict)

In [ ]:
df

,optim,n,log_likelihood,MSE
0,DEOptim,10,-5315.400923,7.310157e-15
1,RandomOptim,10,-6126.819983,2.315093e-14
2,PSOOptim,10,-5315.410680,7.378864e-15
3,DEOptim,50,-26606.716685,3.841156e-17
4,RandomOptim,50,-31657.669063,5.184422e-17
5,PSOOptim,50,-26606.782010,3.869805e-17
6,DEOptim,200,-45663.230583,1.095328e-15
7,RandomOptim,200,-60889.464920,1.165845e-16
8,PSOOptim,200,-45663.960528,1.064659e-15
9,DEOptim,500,-49515.693722,1.344786e-11


In [ ]:
px.line(df, x="n", y="log_likelihood", color="optim")

In [ ]:
px.line(df, x="n", y="MSE", color="optim")